In [1]:
import numpy as np
import pandas as pd

import datetime

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 10

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 200)
        self.fc2 = nn.Linear(200,10)

    def forward(self, x):
        x = self.pool(F.tanh(self.conv1(x)))
        x = self.pool(F.tanh(self.conv2(x)))
        x = torch.flatten(x, 1) 
        x = F.tanh(self.fc1(x))
        x = self.fc2(x)
        return x


net = Net()

criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(net.parameters(), lr=0.0001)
optimizer = optim.Adam(net.parameters(), lr=0.0001)

In [4]:
#writer = SummaryWriter(comment='SGD+tanh')
writer = SummaryWriter(comment='Adam+tanh')

averaging_size = 1000
for epoch in range(25):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # log statistics
        running_loss += loss.item()
        if i % averaging_size == averaging_size-1:    # log every averaging_size mini-batches
             #checking validation data
            correct = 0
            total = 0
            with torch.no_grad():
                for data in testloader:
                    images, labels = data
                    outputs = net(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
            writer.add_scalar("Accuracy/validation", correct/total, epoch * len(trainset)/(batch_size*averaging_size) + (i+1)/averaging_size)
            writer.add_scalar("Loss/train", running_loss/averaging_size, epoch * len(trainset)/(batch_size*averaging_size) + (i+1)/averaging_size)
            writer.flush()
            if i==len(trainloader)-1:
                print(str(f'epoch {epoch + 1}: training loss: {running_loss / averaging_size:.3f}'))
                print(str(f'epoch {epoch + 1}: validation accuracy: {100.0*correct/total:.3f} %'))
            running_loss = 0.0

print('Finished Training')

/opt/conda/lib/python3.9/site-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


epoch 1: training loss: 1.562
epoch 1: validation accuracy: 45.560 %
epoch 2: training loss: 1.443
epoch 2: validation accuracy: 48.770 %
epoch 3: training loss: 1.374
epoch 3: validation accuracy: 50.500 %
epoch 4: training loss: 1.319
epoch 4: validation accuracy: 52.370 %
epoch 5: training loss: 1.262
epoch 5: validation accuracy: 53.530 %
epoch 6: training loss: 1.240
epoch 6: validation accuracy: 54.830 %
epoch 7: training loss: 1.194
epoch 7: validation accuracy: 55.690 %
epoch 8: training loss: 1.164
epoch 8: validation accuracy: 56.500 %
epoch 9: training loss: 1.151
epoch 9: validation accuracy: 57.460 %
epoch 10: training loss: 1.140
epoch 10: validation accuracy: 58.420 %
epoch 11: training loss: 1.102
epoch 11: validation accuracy: 59.130 %
epoch 12: training loss: 1.062
epoch 12: validation accuracy: 59.960 %
epoch 13: training loss: 1.050
epoch 13: validation accuracy: 60.600 %
epoch 14: training loss: 1.025
epoch 14: validation accuracy: 60.900 %
epoch 15: training loss: